In [1]:
import findspark


In [2]:
findspark.init('/home/dhruva_stark/spark-3.4.1-bin-hadoop3')

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession


In [5]:
spark = SparkSession.builder.appName('myproj').getOrCreate()

23/07/09 03:15:06 WARN Utils: Your hostname, linux resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/07/09 03:15:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/09 03:15:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/09 03:15:09 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/07/09 03:15:09 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/07/09 03:15:09 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/07/09 03:15:09 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [6]:
data = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

In [7]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [8]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [9]:
assembler= VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [11]:
output=assembler.transform(data)


In [28]:
final_data=output.select('features','Churn')

In [29]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [30]:
from pyspark.ml.classification import LogisticRegression

In [31]:
lr_churn=LogisticRegression(labelCol='Churn')

In [32]:
fitted_churn_model=lr_churn.fit(train_data)

In [33]:
results = fitted_churn_model.transform(test_data)

In [34]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [35]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Churn')

In [36]:
results.select('Churn','prediction').show()

+-----+----------+
|Churn|prediction|
+-----+----------+
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       1.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 20 rows



In [37]:
AUC = my_eval.evaluate(results)


In [38]:
AUC

0.7201839826839828

In [40]:
final_lr_model=lr_churn.fit(final_data)

In [43]:
new_customers=spark.read.csv('new_customers.csv',inferSchema=True,header=True)

In [44]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [46]:
test_new_customers=assembler.transform(new_customers)


In [47]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [49]:
final_results=final_lr_model.transform(test_new_customers)

In [51]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

